In [2]:
import jieba
from gensim import corpora
from gensim import models
from gensim import similarities

In [3]:
l1 = ["你的名字是什么", "你今年几岁了", "你有多高你胸多大", "你胸多大"]  # 标准问题
a = "你今年多大了"  # 我们的自然语言

s = jieba.cut(a)
print(list(s))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\wanzheng\AppData\Local\Temp\jieba.cache
Loading model cost 0.840 seconds.
Prefix dict has been built succesfully.


['你', '今年', '多大', '了']


### 创建语料库

In [4]:
# 制作语料库
all_doc_list = [list(jieba.cut(doc)) for doc in l1]
print(all_doc_list)

[['你', '的', '名字', '是', '什么'], ['你', '今年', '几岁', '了'], ['你', '有', '多', '高', '你', '胸多大'], ['你', '胸多大']]


In [5]:
# 待匹配文本
doc_test_list = list(jieba.cut(a))

# 制作词袋
dictionary = corpora.Dictionary(all_doc_list)  
# 词袋就是将很多很多的词,进行排列形成一个 词(key) 与一个 标志位(value) 的字典
print("token2id", dictionary.token2id)
print("dictionary", dictionary)

token2id {'什么': 0, '你': 1, '名字': 2, '是': 3, '的': 4, '了': 5, '今年': 6, '几岁': 7, '多': 8, '有': 9, '胸多大': 10, '高': 11}
dictionary Dictionary(12 unique tokens: ['什么', '你', '名字', '是', '的']...)


In [6]:
corpus = [dictionary.doc2bow(doc) for doc in all_doc_list]
# 语料库:
# 这里是将all_doc_list 中的每一个列表中的词语 与 dictionary 中的Key进行匹配
# 得到一个匹配后的结果,例如['你', '今年', '几岁', '了']
# 就可以得到 [[(1, 1), (5, 1), (6, 1), (7, 1)],[123,123,123,123,]]
# 1代表的的是 你 1代表出现一次, 5代表的是 了  1代表出现了一次, 以此类推 6 = 今年 , 7 = 几岁
print(corpus, type(corpus))

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)], [(1, 1), (5, 1), (6, 1), (7, 1)], [(1, 2), (8, 1), (9, 1), (10, 1), (11, 1)], [(1, 1), (10, 1)]] <class 'list'>


In [7]:
#将需要寻找相似度的分词列表 做成 语料库 doc_test_vec
doc_test_vec = dictionary.doc2bow(doc_test_list)
print("doc_test_vec", doc_test_vec, type(doc_test_vec))

doc_test_vec [(1, 1), (5, 1), (6, 1)] <class 'list'>


### 创建Lsi模型

In [13]:
#将corpus语料库(初识语料库) 使用Lsi模型进行训练
lsi = models.LsiModel(corpus)

# 这里的只是需要学习Lsi模型来了解的,这里不做阐述
print("lsi", lsi, type(lsi))

# 语料库corpus的训练结果
print("lsi[corpus]", lsi[corpus] )

lsi LsiModel(num_terms=12, num_topics=200, decay=1.0, chunksize=20000) <class 'gensim.models.lsimodel.LsiModel'>
lsi[corpus] <gensim.interfaces.TransformedCorpus object at 0x0000019F1DB2AAC8>


In [14]:
# 获得语料库doc_test_vec 在 语料库corpus的训练结果 中的 向量表示
print("lsi[doc_test_vec]----", lsi[doc_test_vec])

lsi[doc_test_vec]---- [(0, 0.9910312948854691), (1, 0.06777365757875868), (2, 1.143786647872063), (3, 0.015934342901802574)]


#### 创建稀疏矩阵

In [15]:
# 文本相似度
# 稀疏矩阵相似度 将 主 语料库corpus的训练结果 作为初始值
index = similarities.SparseMatrixSimilarity(
    lsi[corpus], num_features=len(dictionary.keys()))
print("index:", index,'\n')

index: <gensim.similarities.docsim.SparseMatrixSimilarity object at 0x0000019F1DB2AB38> 



#### 查看待匹配文本在稀疏矩阵的表示

In [16]:
# 将 语料库doc_test_vec 在 语料库corpus的训练结果 中的 向量表示 与 语料库corpus的 向量表示 做矩阵相似度计算
sim = index[lsi[doc_test_vec]]
print("sim", sim, type(sim))

sim [0.29518965 0.9900962  0.46673587 0.46673578] <class 'numpy.ndarray'>


#### 查看相似度最匹配的文本

In [17]:
# 对下标和相似度结果进行一个排序,拿出相似度最高的结果
# cc = sorted(enumerate(sim), key=lambda item: item[1],reverse=True)
cc = sorted(enumerate(sim), key=lambda item: -item[1])
print(cc)
high_score = cc[0]
text = l1[high_score[0]]

print(a,'-------', text)
print('相似度：', high_score[1])

[(1, 0.9900962), (2, 0.46673587), (3, 0.46673578), (0, 0.29518965)]
你今年多大了 ------- 你今年几岁了
相似度： 0.9900962


### 创建tfidf模型（作比较）

In [8]:
#将corpus语料库(初识语料库) 使用Tfidf模型进行训练
tfidf = models.TfidfModel(corpus)

print("tfidf", tfidf)

# 语料库corpus的训练结果
print("tfidf[corpus]", tfidf[corpus] )

tfidf TfidfModel(num_docs=4, num_nnz=16)
tfidf[corpus] <gensim.interfaces.TransformedCorpus object at 0x0000019F1DB2A668>


In [9]:
# 获得语料库doc_test_vec 在 语料库corpus的训练结果 中的 向量表示
print("tfidf[doc_test_vec]----", tfidf[doc_test_vec])

tfidf[doc_test_vec]---- [(5, 0.7071067811865475), (6, 0.7071067811865475)]


#### 稀疏矩阵

In [10]:
# 稀疏矩阵相似度 将 主 语料库corpus的训练结果 作为初始值
index = similarities.SparseMatrixSimilarity(
    tfidf[corpus], num_features=len(dictionary.keys()))
print("index:", index,'\n')

index: <gensim.similarities.docsim.SparseMatrixSimilarity object at 0x0000019F1DB2A6D8> 



In [11]:
# 将 语料库doc_test_vec 在 语料库corpus的训练结果 中的 向量表示 与 语料库corpus的 向量表示 做矩阵相似度计算
sim = index[tfidf[doc_test_vec]]
print("sim", sim, type(sim))

sim [0.         0.81649655 0.         0.        ] <class 'numpy.ndarray'>


#### 最相似

In [12]:
cc = sorted(enumerate(sim), key=lambda item: -item[1])
print(cc)
high_score = cc[0]
text = l1[high_score[0]]

print(a,'-------', text)
print('相似度：', high_score[1])

[(1, 0.81649655), (0, 0.0), (2, 0.0), (3, 0.0)]
你今年多大了 ------- 你今年几岁了
相似度： 0.81649655
